### Preamble

In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
# PIP INSTALLS

# for data handling:
#! pip install pandas

# for reading Duration entries:
#! pip install word2number

# for imputing lat/long:
#! pip install geopy

In [49]:
# IMPORTS

import re
import pandas as pd
import datetime as dt

from numpy import select,isnan
from functools import partial
from geopy.geocoders import Nominatim,OpenCage
from geopy.extra.rate_limiter import RateLimiter


# initialise the geopy API
ua = \
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
k = \
"cc6fe73ae2a64a7c937b8f8553dce0a1"
Geogetter = partial(Nominatim(user_agent=ua).geocode , language='en')
#Geogetter = partial(OpenCage(k).geocode , language='en')
Geoget = RateLimiter(Geogetter , min_delay_seconds=1)

In [133]:
# CONSTANTS

# reference states for relocation:
statesUSA = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AS': 'American Samoa',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'DC': 'District of Columbia',
    'FL': 'Florida',
    'GA': 'Georgia',
    'GU': 'Guam',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'MP': 'Northern Mariana Islands',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto Rico',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

statesCND = {
  'AB': 'Alberta',
  'BC': 'British Columbia',
  'MB': 'Manitoba',
  'NB': 'New Brunswick',
  'NL': 'Newfoundland and Labrador',
  'NS': 'Nova Scotia',
  'NT': 'Northwest Territories',
  'NU': 'Nunavut',
  'ON': 'Ontario',
  'PE': 'Prince Edward Island',
  'QC': 'Quebec',
  'SK': 'Saskatchewan',
  'YT': 'Yukon'
}

allStates = {**statesUSA, **statesCND}

# for better shape classification:
shapeMapping = {"light"    :"Flash",
                "circle"   :"Orb",
                "teardrop" :"Teardrop",
                "cigar"    :"Cylinder",
                "disk"     :"Disk",
                "unknown"  :"Indiscernible",
                "oval"     :"Egg",
                "other"    :"Other",
                "sphere"   :"Orb",
                "changing" :"Metamorphing",
                "formation":"Swarm",
                "flash"    :"Flash",
                "chevron"  :"Chevron",
                "0"        :"Orb",
                "triangle" :"Chevron",
                "fireball" :"Flash",
                "cross"    :"Cross",
                "rectangle":"Rectangle",
                "diamond"  :"Diamond",
                "cylinder" :"Cylinder",
                "egg"      :"Egg",
                "cone"     :"Cone"}

### Functions

In [127]:
# PRIVATE

def _readTime(freetext):
    """ Extract datetime text or (value,scale) text pair into pandas.Timedelta format. """
    # not sure how I'll do this. Depends on how cleanupDuration is designed.
    pass


def _countryName(geo):
    name = geo.address.split(', ')[-1]
    return name if 'United States' not in name else 'United States'


def _switchGeo(m,city,abbrev):
    address = city
    if abbrev in allStates.keys():
        address += ", "+allStates[abbrev]

    geo = Geoget(address)
    if not geo:
        print(city)
        return (None,None,None)
    else:
        return {
            3 : ( _countryName(geo) , geo.latitude , geo.longitude ),
            2 : ( None , geo.latitude , geo.longitude ),
            1 : ( _countryName(geo) , None , None )
        }[m]

In [128]:
# PUBLIC

def readDuration(text):
    """ Using NLP-esque procedures to remedy the godless decision to enter duration data as freetext. """
    # each Duration entry is made of any number of components (hopefully just 1 or 2)
    # component is DATETIME (15:38 or 15.38 or 1538) or PAIR (value-scale pair "3 minutes")
    # entry is DICTUM (1 component, e.g. "3 minutes") or SPAN (2 components, "3-5 seconds")
    # SPECIALTIES:
    # "night"/"dusk"/"dawn"     [08:00:00] e.g. "dusk till dawn", "all night"
    # "twenty mins"             [00:20:00] transcribe English to digits
    # "ongoing"/"still going"     [null]   nullify BUT add "ongoing reports"
    # "0"                       [00:00:00] instant
    # DATETIME DICTUM:
    # 18:00                       [null]   nullify
    # 3:00am                      [null]   nullify
    #
    # PAIR DICTUM:
    # 6MINS                     [00:34:00] standardise
    # 3 minutes and 4 seconds   [00:03:04] fuse
    # 3                         [00:03:00] value but no scale given, assume m
    # minutes                   [00:00:05] scale but no value given, assume 5
    # hour                      [01:00:00] detect plurality
    #
    #
    # for SPANS, don't bother with delims as there's too many possibilities.
    # DATETIME SPAN:
    # 18:00 - 18:34             [00:34:00] difference
    #
    # PAIR SPAN
    # 3-5 minutes               [00:04:00] 2v 1s = average
    # 3/5 minutes               [00:04:00] 2v 1s = average
    # 2 seconds - 9 minutes     [00:09:02] 2v 2s = fuse
    #
    #
    # SPECIAL CONSIDERATIONS:
    # 1: 45 mins                [00:01:45] -_-
    # 12 mon                    [00:12:00] "mon" -> "min"
    # entries like "ongoing" can do one. if there's no component as above, it's null to me.
    words = re.split('[\s]' , text)
    dur = words
    return dur


def imputeLocal(states):
    """ More acute country labelling. """
    # For 554 "U.S." entries, a state isn't even given. A lot of these are foreign cities
    #  without even an American town named after them, meaning the label is plainly incorrect.
    #  A few are also Puerto Rican towns, so I included it in the set of (American) States.
    conds = [
        states.isnull(),
        states.isin( set(statesUSA.keys()) ),
        states.isin( set(statesCND.keys()) )
    ]
    choices = [None , 'United States' , 'Canada']        
    return select(conds , choices)


def imputeWeb(missinglist , cities , states):
    overlay = [ _switchGeo( m,city,state ) if m>0 else (None,None,None) for m,city,state in zip(missinglist,cities,states) ]
    return pd.DataFrame( overlay , columns=['Country','Latitude','Longitude'] )

### Mains

In [137]:
if __name__ == '__main__':
    # TASKS:
    #  1
    #  2
    #  3
    #  4
    # (5)
    # ...
    for i in range(8):
        file = 'nuforc'+str(i+1)
        #file = 'sample'
        DF = pd.read_excel( file+'.xlsx' )
        countryCodes = pd.read_excel(
            'country data.xlsx',
            sheet_name='codes'
        )
        
        # clean up dates/times:
        DF['Date'] = DF.Date.dt.date
        #DF['Time'] = DF.Time.replace(-1,None)

        # more acute shape classification:
        DF['Shape'] = DF.Shape.map(shapeMapping).astype('category')

        # impute locally (Country):
        countries_old = DF.Country.copy()
        DF['Country'] = imputeLocal(DF.State)

        # impute off the web (Country, Latitude, Longitude):
        if True:
            toImpute = ['Country','Latitude','Longitude']
            missingnos = DF[toImpute].isnull().sum(axis=1).to_list()
            DF[toImpute] = DF[toImpute].combine_first(
                imputeWeb(missingnos , DF.City.values , DF.State.values)
            )
        
        # reinstate original country if no better guess
        DF['Country'] = DF.Country.combine_first( countries_old )

        # comprehend Duration freetext:
        if False:
            DF['Duration'] = DF.Duration.map(
                readDuration,
                na_action='ignore'
            )

        print(i, "\n" , DF)
        DF.to_excel( file+'_final.xlsx' )

Vacaville or Fairfield
Narraganssett
Cape May Wildwood Crest
Lake Wiley
Short Gap
BAYRAMÃ‡ HACIBEKRLER KÃ–YÃœ 
Karnata Gadak 
West Windsor
SuliszÃ³w, Wietokrzyskie 
Louisvile
Bozra
Monterey Bay Area
Los Gatos Mountains
South Myrtle Beach
Marine park Bklyn
Coolridge
Nicassio
Indian Reservation on way to Barona Casino
Burl County
Fort Olgethorpe
Zanesvill
Mojave
Savanagh
Sanra Rosa
Unspecified location
PHONEX
Woodland hillls
Chesterfiled
Newbrighton
Lake Hamiton
Appointee Falls
Larkesville
Newbrighton
Ballanger County line
Reverside
International waters
Russell to Hays
Everywhere
Pojoque
Shinston
Kissimme
Langely
Cody and Wapiti
Lincoln Boro
Pottstown - Trenton
Lampeter-Strasburg, Reading
Newoort Beach
Bay Minnette
Shokapee
Maison
Trenyon
Christney
Barrow-in-Furness
North Myrtle Beach
Grandmound
New Johnsnville
Chennai
Karmala
Sedonna
Sahuarita
Evertee
Chiraco Summit
In-flight; above AZ
Albatera
Tenterden
Portlamd
Crystal Mt. Ski Resort
Mount Rainier area
Petite Rivier
Abbotsfort
Margate

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Lohman, Missouri',), **{}).
Traceback (most recent call last):
  File "D:\Gilad\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "D:\Gilad\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "D:\Gilad\anaconda3\lib\http\client.py", line 1332, in getresponse
    response.begin()
  File "D:\Gilad\anaconda3\lib\http\client.py", line 303, in begin
    version, status, reason = self._read_status()
  File "D:\Gilad\anaconda3\lib\http\client.py", line 264, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "D:\Gilad\anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "D:\Gilad\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "D:\

Fort Walton Beac
Bismarck double ditch
Cottonwood Lakes, High Sierras
Cresent Lake - Middle of the Mission Mountain Wilderness - F
Floristen
Ogalala
Lexinton
Philladelphia
Myrtal Beach
Entiay
Groton
Nasau bay
Lakeland- Nashville line
Cassendra
Upper Ssaddle River
Lobeland
Tilene
Preson
Unincorporated Miami-Dade County
Goffstown
Drummins
Route 22 bavington exit
Pinallas
Willemstad
New Glarys
Woods Mid-NE
Santa Monica & Brentwood
Bonneylake
Kailua Oahu
Pickneyville
Denvee
Wayneboro
Guanica to Hormigueros 
Bosque farma
MaracanaÃº 
Bizerte
M1 southbound near j8
Porstmouth
Thunderchild
Gulf of Mexico
Frankin
Loganstown
Between Indianola and Canadian
Manchester
Smyna
Riviera Mya 
Porquis Jct.
Samammish
Alex Frasre Bridge
Trinkomalee 
Clarksdale
Coatsville
Autin
Apex-Holly Springs
North utica
Deerfield Twnshp
Hiway 27 and the 6th
Betton
Edmonton HWY 2 Southbound
Pune
Eden Angola
Broke.n arrow
Hicksvllie
Grandhaven
Blueash
Mcenry
Mount Gildead
Silverhilll
DÃºbrava, Å½ilinskÃ½ kraj  
Raniganj/P

Medford and Murphy
Solome
East Windsor
1 
              Date      Time           City State        Country      Shape  \
0      2019-12-12  18:43:00        Chester    VA  United States      Flash   
1      2019-03-22  18:30:00     Rocky Hill    CT  United States        Orb   
2      2019-04-17  02:00:00         Ottawa    ON         Canada   Teardrop   
3      2009-03-15  18:00:00         Peoria    NY  United States   Cylinder   
4      2019-04-02  20:25:00     Kirbyville    TX  United States       Disk   
...           ...       ...            ...   ...            ...        ...   
10835  2006-09-01  01:00:00       Kirkland    WA  United States  Rectangle   
10836  2006-09-02  03:02:00          Craig    CO  United States        NaN   
10837  2006-09-03  18:22:00       Aalborg    NaN        Denmark        Orb   
10838  2006-09-08  20:00:00        Berwick    ME  United States        Egg   
10839  2006-09-10  21:00:00  Lake Elsinore    CA  United States        Orb   

               Durat

New Hatford
Red Bluff - Dribble Creek Subdivision
Sierra Nevada Mountains
I-10 eastbound @ markers 2-65
Krasdnoyarsk 
McGregor range
Pomoma
Viby Sjealland 
Yukon Territory
Toppsey
Othello-Pullman
Brigantine
Boonton Town
Volujntown
Panama City Beach
Oak Shores Lake Berryessa
Bellville
Ralleigh
Berkley Twsp.
Eaugalle
KlaksvÃ­k 
Cape May Courthouse
Somerviller
not in one
Llandudnol 
Jounieh-Jabal Lebnen 
Sarnia
USS John F. Kennedy 
Corral Hollow road vicinity
Cape May Courthouse
Cumberland to Freeport area
Litha
Enerson
Muskokee
Campground Kendallville
unsure exact name
Allegheny State Park
Sorento
OujÃ© Bougoumou
In-flight sighting
Pushkarmoe 
Machedney Park
Gracville
Pounchatola
Machu Piccu 
New Bedfor
Mainville
Antioch Community
Alpheretta
Pollockpines
Wipholt
ZIP code 20876
Leeds
Wurtand
russlville
Cataqua
Achusnet
Granview
Linoir City
Swindon/Didcott 
Williamslake
New Kensignton
Close to Wellton
Feltham
Acusnet
Iochwinnoch 
Butte & Bozeman
Mohawk Valley Mtns.
Eugen
RzeszÃ³w 
Melbrook

...

Alejuela 
N FL; S GA
Fall River
Piranguinho
Sea Isle
Caxi/Luang Prahbang 
Yellanhalli-643243 
Wildlife Refuge, Medicine Park, Ft. Sill
Tomaball
Belbeis 
Loon Mountain Ski Club
South Myrtle Beach Pier
Millersville
DeMotte-Roselawn
Milk Hill, Alton 
Berkley Heights
Visalia-Lemoore
Lexingtob
Mornas A7 Motorway 
Peabody-Saugus
Murfeesboro
Jasonvill
Jasonvill
North Massapequa
Ashgrove
mile marker 41 garden state parkway
Stering
LÃ©vis
StrÃ¡Å¾ pod Ralskem 
In-flight sighting
Red Feather area
Port Jeff Station
Doha
Severa Park
Brick Town
Mapillville
Lexingon Heights
Hummlestown
Greenbried
Steinkjer, Nord-TrÃ¸ndelag 
Mechaniscville
Camp Seph Mack
Alaska cruise
Larnca 
Space station video
Berkley Heights
Ojibways of Onigaming FN
LuleÃ¥ 
Monett
North Massapequa
Soccoro
Pine Lake Leisure Center
Tunapuna
New Symrna beach
Queenbeyan 
Conway and Economy Boro
Bloomignton
Chicao
Schwencksville
Hunstville
West Natcoke
Monee Manhattan Rd, Monee,on i57 headed north IL 60449
Sundrland 
KecskemÃ©t 
Cutyhun

RateLimiter caught an error, retrying (0/2 tries). Called with (*('I-95 south, South Carolina',), **{}).
Traceback (most recent call last):
  File "D:\Gilad\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "D:\Gilad\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "D:\Gilad\anaconda3\lib\http\client.py", line 1332, in getresponse
    response.begin()
  File "D:\Gilad\anaconda3\lib\http\client.py", line 303, in begin
    version, status, reason = self._read_status()
  File "D:\Gilad\anaconda3\lib\http\client.py", line 264, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "D:\Gilad\anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "D:\Gilad\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
 

RateLimiter swallowed an error after 2 retries. Called with (*('I-95 south, South Carolina',), **{}).
Traceback (most recent call last):
  File "D:\Gilad\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 426, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "D:\Gilad\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 421, in _make_request
    httplib_response = conn.getresponse()
  File "D:\Gilad\anaconda3\lib\http\client.py", line 1332, in getresponse
    response.begin()
  File "D:\Gilad\anaconda3\lib\http\client.py", line 303, in begin
    version, status, reason = self._read_status()
  File "D:\Gilad\anaconda3\lib\http\client.py", line 264, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "D:\Gilad\anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "D:\Gilad\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  Fi

I-95 south
Kissimme
Ridge Mountain Elkins
Kuparuk Oilfield
South of Sarasota
Stewarts town
Glenallen
Supulpa
5 
              Date      Time                City State        Country    Shape  \
0      2013-03-05  18:15:00         Bakersfield    CA  United States    Flash   
1      2013-03-05  20:00:00              Normal    IL  United States  Chevron   
2      2013-03-05  20:00:00  San Jose del Cabo    NaN         Mexico    Flash   
3      2013-08-02  01:15:00       Franklinville    NJ  United States      Egg   
4      2013-08-02  01:30:00             Orlando    FL  United States    Flash   
...           ...       ...                 ...   ...            ...      ...   
10835  2014-07-04  23:00:00             Chicago    IL  United States    Flash   
10836  2014-07-04  23:00:00          Brookfield    IL  United States  Diamond   
10837  2014-07-04  23:00:00              Eugene    OR  United States      NaN   
10838  2014-07-04  23:00:00          Fort Wayne    IN  United States    Flash

Drove/Soller 
Marrietta
Rockvile
Ouchita National Forest
Olmsteadville
Frostburg, Lavale
Bistib
Vannuys
Sicklerviller
Some where between Fernley and shurz
Gonzales
Tuscarora Indian Reservation
Sears Point Raceway
Anderson or Oconee
Willglen
Bahgram 
Wynanskill
Saint Fracois Xavier
Com/Lugano 
Encintas
Mismisburg
Eastaboba
Unspecified
Chihlowie
Aloharetta
Manoa Hills
Groom Creek wilderness area
JÃ¶nkÃ¶ping 
Waldoboro
Woods Canyon Lake
Slippery Rock Township, near New Castle
Roasharon
Rhien
Saint Clair Shored
El Soreno
Hendersom
Old Town Katy
Weverville
Connelsville
Birmigham 
Dorset
Gulf of Mexico
Lexinton
Palm Spings
Rather not say.
Baldwin Borough
Mayfeld
Groton Long Point
Vellore/Hyderabad/Bangalore
VÃ¤sterÃ¥s 
Phalaborwa
McHenrry
Levene
Curchorem, Savordem 
Joppatown
Bolivar, Hermitage, and Urbana
Ran home Santa Fe
Ocala
West Farmington
Wheeler AAF
Antinito
Lake Wataree
Lakveville
Ocean & Monmouth Counties
Hartebeespoortdam 
DeLuz
Harlansburgh
Southwest UK
Attelboro
Musquodoboit Har